In [ ]:
# from google.colab import drive
!wget -q https://www.dropbox.com/s/bew0mp12w3eqss8/MCTest.zip
!unzip -o /content/MCTest.zip 

Archive:  /content/MCTest.zip
  inflating: __MACOSX/._MCTest       
  inflating: MCTest/.DS_Store        
  inflating: __MACOSX/MCTest/._.DS_Store  
  inflating: __MACOSX/MCTest/._MCTest  
  inflating: __MACOSX/MCTest/._MCTestAnswers  
  inflating: MCTest/MCTest/mc160.train.txt  
  inflating: __MACOSX/MCTest/MCTest/._mc160.train.txt  
  inflating: MCTest/MCTest/mc160.dev.txt  
  inflating: __MACOSX/MCTest/MCTest/._mc160.dev.txt  
  inflating: MCTest/MCTest/mc160.test.tsv  
  inflating: __MACOSX/MCTest/MCTest/._mc160.test.tsv  
  inflating: MCTest/MCTest/mc160.train.ans  
  inflating: __MACOSX/MCTest/MCTest/._mc160.train.ans  
  inflating: MCTest/MCTest/mc500.train.tsv  
  inflating: __MACOSX/MCTest/MCTest/._mc500.train.tsv  
  inflating: MCTest/MCTest/mc160.dev.ans  
  inflating: __MACOSX/MCTest/MCTest/._mc160.dev.ans  
  inflating: MCTest/MCTest/mc160.test.txt  
  inflating: __MACOSX/MCTest/MCTest/._mc160.test.txt  
  inflating: MCTest/MCTest/mc500.train.txt  
  inflating: __MACOSX/MC

## Import lib

In [ ]:
! git clone https://github.com/NVIDIA/apex.git
% cd apex
! pip install -v --no-cache-dir ./
%cd ..

from apex import amp

import pandas as pd
import numpy as np
import logging
import argparse
import random
import numpy as np
import os
import time
import glob
import json

!pip install transformers
!pip install sentencepiece
from transformers import BertTokenizer, BertForMultipleChoice, BertConfig, AdamW, get_linear_schedule_with_warmup
from transformers import DistilBertConfig, DistilBertTokenizer, DistilBertForMultipleChoice
from transformers import AlbertConfig, AlbertTokenizer, AlbertForMultipleChoice


import torch
from torch.optim.adam import Adam
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from torch.utils.data.distributed import DistributedSampler
from torch.nn.utils.rnn import pad_sequence

fatal: destination path 'apex' already exists and is not an empty directory.
/content/apex
Created temporary directory: /tmp/pip-ephem-wheel-cache-ij6ne1gy
Created temporary directory: /tmp/pip-req-tracker-yzg_yeyt
Created requirements tracker '/tmp/pip-req-tracker-yzg_yeyt'
Created temporary directory: /tmp/pip-install-lmptia6q
Processing /content/apex
  Created temporary directory: /tmp/pip-req-build-54p4ej83
  Added file:///content/apex to build tracker '/tmp/pip-req-tracker-yzg_yeyt'
    Running setup.py (path:/tmp/pip-req-build-54p4ej83/setup.py) egg_info for package from file:///content/apex
    Running command python setup.py egg_info


    torch.__version__  = 1.8.1+cu101


    running egg_info
    creating /tmp/pip-req-build-54p4ej83/pip-egg-info/apex.egg-info
    writing /tmp/pip-req-build-54p4ej83/pip-egg-info/apex.egg-info/PKG-INFO
    writing dependency_links to /tmp/pip-req-build-54p4ej83/pip-egg-info/apex.egg-info/dependency_links.txt
    writing top-level names to /tmp/

## Bert Pre-Trained Tokenizers

In [ ]:
def tokenization(): 
    # tokenizer = BertTokenizer.from_pretrained('bert-large-uncased')
    tokenizer = AlbertTokenizer.from_pretrained('albert-base-v2')
    return tokenizer

## Bert Pre-Trained Model

In [ ]:
def mctest_model():
    # config = BertConfig.from_pretrained('bert-large-uncased', num_labels=4)
    # model = BertForMultipleChoice.from_pretrained('bert-large-uncased', config=config)
    config = AlbertConfig.from_pretrained('albert-base-v2', num_labels=4)
    model = AlbertForMultipleChoice.from_pretrained('albert-base-v2', config=config)
    return model

## MCTest Features Class

In [ ]:
class InputMCTestFeatures(object):
    def __init__(self,
                 example_id,
                 choices_features,
                 label):
        self.example_id = example_id
        self.choices_features = [
            {
                'input_ids': input_ids,
                'segment_ids': segment_ids,
                'input_mask': input_mask
            }
            for _, input_ids, segment_ids, input_mask in choices_features
        ]
        self.label = label

## MCTest Object Class

In [ ]:
class mctest(object):
    def __init__(self,
                 test_id,
                 article,
                 start_ending,
                 ending_1,
                 ending_2,
                 ending_3,
                 ending_4,
                 label):
        self.test_id = test_id
        self.article = article
        self.start_ending = start_ending
        self.endings = [
            ending_1,
            ending_2,
            ending_3,
            ending_4,
        ]
        self.label = label

    def __str__(self):
        return self.__repr__()

    def __repr__(self):
        l = [
            "id: {}".format(self.test_id),
            "article: {}".format(self.article),
            "question: {}".format(self.start_ending),
            "option_1: {}".format(self.endings[0]),
            "option_2: {}".format(self.endings[1]),
            "option_3: {}".format(self.endings[2]),
            "option_4: {}".format(self.endings[3]),
        ]


        if self.label is not None:
            l.append("label: {}".format(self.label))

        return ", ".join(l)

## Prepare MCTest data

In [ ]:
def replace_article_str(df):
  str_lst = ['\\\\newline\\\\newline', '\\n', '\\', 'newline']
  for i in str_lst:
    df["article"] = df["article"].str.replace(i, " ", regex=True)
  return df

In [ ]:
 def mc_train():
   train_lst = []
   for i in ["160", "500"]:
     mc_train_question = pd.read_csv(
          '/content/MCTest/MCTest/mc{}.train.tsv'.format(i),
          sep='\t',
          header=None,
          names=["id", "author-worktimes", "article",
                "q1", "q1_c1", "q1_c2", "q1_c3", "q1_c4",
                "q2", "q2_c1", "q2_c2", "q2_c3", "q2_c4",
                "q3", "q3_c1", "q3_c2", "q3_c3", "q3_c4",
                "q4", "q4_c1", "q4_c2", "q4_c3", "q4_c4", ])
     mc_train_ans = pd.read_csv(
        '/content/MCTest/MCTest/mc{}.train.ans'.format(i),
        sep='\t',
        header=None,
        names=['q1_ans', 'q2_ans', 'q3_ans', 'q4_ans', ])   
     # Dropping id and author-worktimes columns
     mc_train = mc_train_question.drop(columns=["id", "author-worktimes"])
     mc_train = mc_dataset_pre(mc_train, mc_train_ans)
     mc_train = replace_article_str(mc_train)
    #  mc_train['article'] = mc_train['article'].str.replace("\\n"," ")
    #  mc_train['article'] = mc_train['article'].str.replace("\\\\newline"," ")
     train_lst.append(mc_train)
   final_train_df = pd.concat(train_lst, ignore_index=True)
   return final_train_df

def mc_dev():
   dev_lst = []
   for i in ["160", "500"]: 
      mc_dev_question = pd.read_csv(
          '/content/MCTest/MCTest/mc{}.dev.tsv'.format(i),
          sep='\t',
          header=None,
          names=["id", "author-worktimes", "article",
                 "q1", "q1_c1", "q1_c2", "q1_c3", "q1_c4",
                 "q2", "q2_c1", "q2_c2", "q2_c3", "q2_c4",
                 "q3", "q3_c1", "q3_c2", "q3_c3", "q3_c4",
                 "q4", "q4_c1", "q4_c2", "q4_c3", "q4_c4", ])
      mc_dev_ans = pd.read_csv(
          '/content/MCTest/MCTest/mc{}.dev.ans'.format(i),
          sep='\t',
          header=None,
          names=['q1_ans', 'q2_ans', 'q3_ans', 'q4_ans', ])     
      # Dropping id and author-worktimes columns
      mc_dev = mc_dev_question.drop(columns=["id", "author-worktimes"])
      mc_dev = mc_dataset_pre(mc_dev, mc_dev_ans)
      mc_dev = replace_article_str(mc_dev)
      # mc_dev['article'] = mc_dev['article'].str.replace("\\"," ")
      # mc_dev['article'] = mc_dev['article'].str.replace("\ newline"," ")
      dev_lst.append(mc_dev)
   final_dev_df = pd.concat(dev_lst, ignore_index=True)
   return final_dev_df
 

 def mc_test():
   test_lst = []
   for i in ["160", "500"]:
      mc_test_question = pd.read_csv(
          '/content/MCTest/MCTest/mc{}.test.tsv'.format(i),
          sep='\t',
          header=None,
          names=["id", "author-worktimes", "article",
                 "q1", "q1_c1", "q1_c2", "q1_c3", "q1_c4",
                 "q2", "q2_c1", "q2_c2", "q2_c3", "q2_c4",
                 "q3", "q3_c1", "q3_c2", "q3_c3", "q3_c4",
                 "q4", "q4_c1", "q4_c2", "q4_c3", "q4_c4", ])
      mc_test_ans = pd.read_csv(
          '/content/MCTest/MCTestAnswers/mc{}.test.ans'.format(i),
          sep='\t',
          header=None,
          names=['q1_ans', 'q2_ans', 'q3_ans', 'q4_ans', ])     
      # Dropping id and author-worktimes columns
      mc_test = mc_test_question.drop(columns=["id", "author-worktimes"])
      mc_test = mc_dataset_pre(mc_test, mc_test_ans)
      mc_test = replace_article_str(mc_test)
      # mc_test['article'] = mc_test['article'].str.replace("\n"," ")
      # mc_test['article'] = mc_test['article'].str.replace("\ newline"," ")
      test_lst.append(mc_test)
   final_test_df = pd.concat(test_lst, ignore_index=True)
   return final_test_df

In [ ]:
def mc_dataset_pre(df, df_ans):
    # Adding new column of the required sentence type of the answer
    for i in range(1, 5):
        df.insert(loc=df.columns.get_loc("q{}".format(str(i))) + 1,
                  column="q{}_ans_sen_type".format(i),
                  value="")
    # Checking what sentence type is required for finding the answer
    for i in range(1, 5):
        for k in range(df.shape[0]):
            if "multiple:" in df.loc[k, "q{}".format(str(i))].split()[0]:
                df.loc[k, "q{}_ans_sen_type".format(i)] = "multiple"
                df.loc[k, "q{}".format(str(i))] = df.loc[k, "q{}".format(str(i))].replace("multiple:", "")
            elif "one:" in df.loc[k, "q{}".format(str(i))].split()[0]:
                df.loc[k, "q{}_ans_sen_type".format(i)] = "one"
                df.loc[k, "q{}".format(str(i))] = df.loc[k, "q{}".format(str(i))].replace("one:", "")

    # Adding a new column of the question answer
    for i in range(1, 5):
        df.insert(loc=df.columns.get_loc("q{}_c4".format(str(i))) + 1,
                    column="q{}_ans".format(i),
                    value=df_ans.iloc[:, i - 1])

    # Adding a new column of the text of the question answer
    for i in range(1, 5):
        df.insert(loc=df.columns.get_loc("q{}_ans".format(str(i))) + 1,
                  column="q{}_ans_text".format(i),
                  value="")

    # Putting the corresponding answer of the question into the dataframe
    for i in range(1, 5):
        for k in range(df.shape[0]):
            if df.loc[k, "q{}_ans".format(str(i))] == "A":
                df.loc[k, "q{}_ans_text".format(str(i))] = df.loc[k, "q{}_c1".format(str(i))]
            elif df.loc[k, "q{}_ans".format(str(i))] == "B":
                df.loc[k, "q{}_ans_text".format(str(i))] = df.loc[k, "q{}_c2".format(str(i))]
            elif df.loc[k, "q{}_ans".format(str(i))] == "C":
                df.loc[k, "q{}_ans_text".format(str(i))] = df.loc[k, "q{}_c3".format(str(i))]
            elif df.loc[k, "q{}_ans".format(str(i))] == "D":
                df.loc[k, "q{}_ans_text".format(str(i))] = df.loc[k, "q{}_c4".format(str(i))]

    # Get the column of article, question, answer sentence type and answer from training dataset
    # Put them into a final training dataset
    df_final = pd.DataFrame(columns=["article", "question", "option_1", "option_2", "option_3", "option_4",
                                     "ans_sen_type", "answer"])
    for art in range(df.shape[0]):
        for q in range(1, 5):
            df_final = df_final.append(pd.DataFrame([[df.loc[art, "article"],
                                                      df.loc[art, "q{}".format(str(q))],
                                                      df.loc[art, "q{}_c1".format(str(q))],
                                                      df.loc[art, "q{}_c2".format(str(q))],
                                                      df.loc[art, "q{}_c3".format(str(q))],
                                                      df.loc[art, "q{}_c4".format(str(q))],
                                                      df.loc[art, "q{}_ans_sen_type".format(str(q))],
                                                      df.loc[art, "q{}_ans".format(str(q))]
                                                      # df.loc[art, "q{}_ans_text".format(str(q))]
                                                      ]],
                                                    columns=["article", "question", "option_1", "option_2", "option_3",
                                                             "option_4",
                                                             "ans_sen_type", "answer"]),
                                       ignore_index=True)
    return df_final

## MCTest Dataset

In [ ]:
def mctestDataSet(tokenizer, task_type, max_seq_length):
    df = None
    if task_type == "train":
        df = mc_train()
    elif task_type == "dev":
        df = mc_dev()
    elif task_type == "test":
        df = mc_test()
  
    examples = []
    for i in range(df.shape[0]):
        answer = ord(df.loc[i, "answer"]) - ord('A')
        examples.append(mctest(
            test_id="{}_{}-{}".format("mc", task_type, i),
            article=df.loc[i, "article"],
            start_ending=df.loc[i, "question"],
            
            ending_1=df.loc[i, "option_1"],
            ending_2=df.loc[i, "option_2"],
            ending_3=df.loc[i, "option_3"],
            ending_4=df.loc[i, "option_4"],
            label=answer,
        ))
    features = []
    for example_index, example in enumerate(examples):
        # article tokens
        article_tokens = tokenizer.tokenize(example.article)
        # quesiton tokens
        start_ending_tokens = tokenizer.tokenize(example.start_ending)

        choices_features = []
        for ending_index, ending in enumerate(example.endings):
            article_tokens_choice = article_tokens[:]
            ending_tokens = start_ending_tokens + ["[SEP]"] + tokenizer.tokenize(ending)

            encode = tokenizer.encode_plus(article_tokens_choice, 
                                           ending_tokens,
                                           add_special_tokens=True,
                                           padding='max_length',
                                           truncation=True,
                                           max_length=max_seq_length)
            
            input_ids = encode["input_ids"]
            segments_ids = encode["token_type_ids"]
            input_mask = encode["attention_mask"]
            tokens = tokenizer.decode(input_ids)
            # print(tokens)

            assert len(input_ids) == max_seq_length
            assert len(input_mask) == max_seq_length
            assert len(segments_ids) == max_seq_length

            choices_features.append((tokens, input_ids, segments_ids, input_mask))

        label = example.label
        features.append(InputMCTestFeatures(
            example_id=example.test_id,
            choices_features=choices_features,
            label=label
        ))

    tensor_input_ids = torch.tensor([[choice["input_ids"] for choice in feature.choices_features] for feature in features], dtype=torch.long)
    tensor_segment_ids = torch.tensor([[choice["segment_ids"] for choice in feature.choices_features] for feature in features], dtype=torch.long)
    tensor_input_mask = torch.tensor([[choice["input_mask"] for choice in feature.choices_features] for feature in features], dtype=torch.long)
    tensor_labels = torch.tensor([f.label for f in features], dtype=torch.long)   
    tensor_dataset = TensorDataset(tensor_input_ids, tensor_segment_ids, tensor_input_mask, tensor_labels)  
      
    if task_type == "test":
      new_test_data = []
      for i in tensor_dataset:
        new_input_ids = i[0]
        new_segment_ids = i[1]
        new_input_masks = i[2]
        # new_labels = None
        # new_test_data.append([new_input_ids, new_segment_ids, new_input_masks, new_labels])
        new_test_data.append([new_input_ids, new_segment_ids, new_input_masks])
      tensor_dataset = tuple(new_test_data)

    return tensor_dataset

## Defining GPU

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
!nvidia-smi

Sat Apr 10 10:51:10 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.67       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   56C    P0    31W / 250W |      2MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Prediction


In [ ]:
def get_predictions(model, dataloader, compute_acc=False):
    predictions = None
    correct = 0
    total = 0
  
    with torch.no_grad():
        for data in dataloader:
            if next(model.parameters()).is_cuda:
                data = [t.to(device) for t in data if t is not None]
            
            tokens_tensors, segments_tensors, masks_tensors = data[:3]
            outputs = model(input_ids=tokens_tensors, 
                            token_type_ids=segments_tensors, 
                            attention_mask=masks_tensors)
            
            # print(outputs, "\n")
            logits = outputs[0]
            # print("logits: {}".format(logits))
                        
            _, pred = torch.max(logits.data, 1)
            
            if compute_acc:
                labels = data[3]
                total += labels.size(0)
                correct += (pred == labels).sum().item()
                
            if predictions is None:
                predictions = pred
            else:
                predictions = torch.cat((predictions, pred))
    
    if compute_acc:
        acc = correct / total
        return predictions, acc
    return predictions

## Run MCTest

In [ ]:
BATCH = 8
MAX_SEQ_LEN = 128
LR = 2e-5

tokenizer = tokenization()
model = mctest_model()
model = model.to(device)

train_data = mctestDataSet(tokenizer, "train", MAX_SEQ_LEN)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=BATCH)
  
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]
adam_optimizer = AdamW(optimizer_grouped_parameters, lr=LR)
model, adam_optimizer = amp.initialize(model, adam_optimizer, opt_level="O1")



Some weights of the model checkpoint at albert-base-v2 were not used when initializing AlbertForMultipleChoice: ['predictions.bias', 'predictions.LayerNorm.weight', 'predictions.LayerNorm.bias', 'predictions.dense.weight', 'predictions.dense.bias', 'predictions.decoder.weight', 'predictions.decoder.bias']
- This IS expected if you are initializing AlbertForMultipleChoice from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing AlbertForMultipleChoice from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of AlbertForMultipleChoice were not initialized from the model checkpoint at albert-base-v2 and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on

Selected optimization level O1:  Insert automatic casts around Pytorch functions and Tensor methods.

Defaults for this optimization level are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic
Processing user overrides (additional kwargs that are not None)...
After processing overrides, optimization options are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic


In [ ]:
EPOCHS = 3
print("Start Training...")
for epoch in range(EPOCHS):
    epoch_start = time.time()
    train_loss = 0.0
    t0_epoch, t0_batch = time.time(), time.time()
    total_loss, batch_loss, batch_counts = 0, 0, 0
    model.train()
    # for data in train_dataloader:
    for data in train_dataloader:
      tokens_tensors, segments_tensors, masks_tensors, labels_tensor = [t.to(device) for t in data]
      
      # Set the gradient to 0
      adam_optimizer.zero_grad()
      outputs = model(input_ids=tokens_tensors, 
                        token_type_ids=segments_tensors, 
                        attention_mask=masks_tensors, 
                        labels=labels_tensor)

      loss = outputs[0]

      train_loss += loss.item()
      with amp.scale_loss(loss, adam_optimizer) as scaled_loss:
        scaled_loss.backward()
      adam_optimizer.step()

    # Calculating Accuracy
    _, train_accuracy = get_predictions(model, train_dataloader, compute_acc=True)

    ## Evaluation
    model.eval()    
    tokenizer = tokenization()
    dev_data = mctestDataSet(tokenizer=tokenizer, task_type="dev", max_seq_length=MAX_SEQ_LEN)
    dev_dataloader = DataLoader(dev_data, batch_size=BATCH)   
    # Tracking variables
    dev_accuracy = []
    dev_loss = []   
    eval_time_start = time.time()
    # For each batch in our validation set...
    for data in dev_dataloader:
        # Load batch to GPU
        tokens_tensors, segments_tensors, masks_tensors, labels_tensor = [t.to(device) for t in data]   
        # Compute logits
        with torch.no_grad():
          outputs = model(input_ids=tokens_tensors, 
                      token_type_ids=segments_tensors, 
                      attention_mask=masks_tensors, 
                      labels=labels_tensor)
        loss = outputs[0]
        logits = outputs[1]
        # Compute loss
        dev_loss.append(loss.item())    
        # Get the predictions
        preds = torch.argmax(logits, dim=1).flatten()   
        # Calculating Accuracy

    _, dev_acc = get_predictions(model, dev_dataloader, compute_acc=True)
    dev_accuracy.append(dev_acc)
    eval_time_stop = time.time()    
    # Compute the average accuracy and loss over the validation set.
    dev_loss = np.mean(dev_loss)
    dev_accuracy = np.mean(dev_accuracy)
    
    epoch_end = time.time()   
    print("Training Epoch: {}, TRAIN-Loss: {}, TRAIN-Accuracy: {}, DEV-Loss: {}, DEV-Accuracy: {},Epcoh Time: {}".
          format(epoch+1, train_loss, train_accuracy, dev_loss, dev_accuracy, epoch_end-epoch_start))

Start Training...
Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 32768.0
Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 16384.0
Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 8192.0
Training Epoch: 1, TRAIN-Loss: 227.33915901184082, TRAIN-Accuracy: 0.6297297297297297, DEV-Loss: 1.031173864006996, DEV-Accuracy: 0.571875,Epcoh Time: 126.07585573196411
Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 4096.0
Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 2048.0
Training Epoch: 2, TRAIN-Loss: 165.5555994808674, TRAIN-Accuracy: 0.7155405405405405, DEV-Loss: 0.9813106685876847, DEV-Accuracy: 0.571875,Epcoh Time: 125.93611097335815
Training Epoch: 3, TRAIN-Loss: 120.9088600948453, TRAIN-Accuracy: 0.8648648648648649, DEV-Loss: 0.9670368945226073, DEV-Accuracy: 0.609375,Epcoh Time: 126.0745460987091


## Saving Model

In [ ]:
torch.save(model.state_dict(), 'checkpoint.pth')

## Loading Model

In [ ]:
state_dict = torch.load('checkpoint.pth')
print(state_dict.keys())

odict_keys(['albert.embeddings.position_ids', 'albert.embeddings.word_embeddings.weight', 'albert.embeddings.position_embeddings.weight', 'albert.embeddings.token_type_embeddings.weight', 'albert.embeddings.LayerNorm.weight', 'albert.embeddings.LayerNorm.bias', 'albert.encoder.embedding_hidden_mapping_in.weight', 'albert.encoder.embedding_hidden_mapping_in.bias', 'albert.encoder.albert_layer_groups.0.albert_layers.0.full_layer_layer_norm.weight', 'albert.encoder.albert_layer_groups.0.albert_layers.0.full_layer_layer_norm.bias', 'albert.encoder.albert_layer_groups.0.albert_layers.0.attention.query.weight', 'albert.encoder.albert_layer_groups.0.albert_layers.0.attention.query.bias', 'albert.encoder.albert_layer_groups.0.albert_layers.0.attention.key.weight', 'albert.encoder.albert_layer_groups.0.albert_layers.0.attention.key.bias', 'albert.encoder.albert_layer_groups.0.albert_layers.0.attention.value.weight', 'albert.encoder.albert_layer_groups.0.albert_layers.0.attention.value.bias', 'a

In [ ]:
model = mctest_model()
model = model.to(device)
model.load_state_dict(state_dict)

Some weights of the model checkpoint at albert-base-v2 were not used when initializing AlbertForMultipleChoice: ['predictions.bias', 'predictions.LayerNorm.weight', 'predictions.LayerNorm.bias', 'predictions.dense.weight', 'predictions.dense.bias', 'predictions.decoder.weight', 'predictions.decoder.bias']
- This IS expected if you are initializing AlbertForMultipleChoice from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing AlbertForMultipleChoice from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of AlbertForMultipleChoice were not initialized from the model checkpoint at albert-base-v2 and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on

<All keys matched successfully>

## Test Prediction

In [ ]:
TEST_MAX_SEQ_LEN = 128
TEST_BATCH = 8

model.eval()
tokenizer = tokenization()

test_data = mctestDataSet(tokenizer=tokenizer, task_type="test", max_seq_length=TEST_MAX_SEQ_LEN)

test_dataloader = DataLoader(test_data, batch_size=TEST_BATCH)

predictions = get_predictions(model, test_dataloader)

# Testing Result Evaluation
from sklearn.metrics import accuracy_score

label_map = {'A': 0, 'B': 1, 'C': 2, 'D': 3}
index_map = {v: k for k, v in label_map.items()}

df = pd.DataFrame({"pre_answer": predictions.tolist()})
df['pre_answer'] = df.pre_answer.apply(lambda x: index_map[x])
df_pred = pd.concat([mc_test().loc[:, ["article"]], 
                     mc_test().loc[:, ["question"]], 
                     mc_test().loc[:, ["answer"]], 
                     df.loc[:, 'pre_answer']], axis=1)

list_true = df_pred.loc[:, ["answer"]].values.tolist()
list_pre = df_pred.loc[:, ["pre_answer"]].values.tolist()

print("Testing Accuracy: {}".format(accuracy_score(list_true, list_pre)))

Testing Accuracy: 0.6023809523809524


In [ ]:
df_pred

,article,question,answer,pre_answer
0,Todd is a small boy in the town of Rocksville....,How long was it before Todd made it to the rock?,A,C
1,Todd is a small boy in the town of Rocksville....,What did Todd have to do so his dad would wat...,A,D
2,Todd is a small boy in the town of Rocksville....,What was Todd's favorite part of Lake Keet?,B,B
3,Todd is a small boy in the town of Rocksville....,How did Todd's dad celebrate Todd's hard work?,B,A
4,Mortamer was a tree monkey who lived in the ju...,What scared Mortamer?,D,D
...,...,...,...,...
835,Greg and his mother were building a racing car...,Where was the race happening?,B,B
836,Joey went to a baseball game during the winter...,Who went to the baseball game and with how ma...,A,D
837,Joey went to a baseball game during the winter...,what kind of store did Joey turn into?,D,A
838,Joey went to a baseball game during the winter...,Which team won the game Joey went to and by h...,D,A
